# Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Above we connected to the SQLite facebook database file

# Data Overview

Let's get a sense of what the data looks like.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Above is inofrmation on any tables in the database.

In [3]:
%%sql
Select *
From facts
Limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Above are the first 5 rows from the facts table, with all columns.

# Summary Statistics

In [4]:
%%sql
SELECT MIN(population), 
MAX(population),
MIN(population_growth),
MAX (population_growth)
FROM facts;


Done.


MIN(population),MAX(population),MIN(population_growth),MAX (population_growth)
0,7256490011,0.0,4.02


This shows the:  
Minimum population  
Maximum population  
Minimum population growth  
Maximum population growth  

# Exploring Outliers

In [5]:
%%sql
Select name, population
From facts
Where population = 0 or population = 7256490011;

Done.


name,population
Antarctica,0
World,7256490011


The population of 0 is for Antartica, while the highest population is for the World, so we should exclude this.

# Summary Statistics Revisited

In [7]:
%%sql
SELECT MIN(population), 
MAX(population),
MIN(population_growth),
MAX (population_growth)
FROM facts
WHERE name is not 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX (population_growth)
0,1367485388,0.0,4.02


Above shows the max population of a country, not including World.  
Lets look at the average population and average area below.

In [8]:
%%sql
Select AVG(population), AVG(area)
FROM facts
WHERE name is not 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


## Finding Densely Populated Countries

We want to find countries that have above average population and below average area.

In [15]:
%%sql
SELECT name, population, area, 
CAST(area as FLOAT)/CAST(population as FLOAT) as density
FROM facts
WHERE name is not 'World' and
      population > (Select AVG(population)
                    FROM facts
                    WHERE name is not 'World') and
      area < (Select AVG(area)
              FROM facts
              WHERE name is not 'World')
ORDER BY density ASC
;
    

Done.


name,population,area,density
Bangladesh,168957745,148460,0.0008786812347667163
"Korea, South",49115196,99720,0.002030328861967689
Philippines,100998376,300000,0.002970344790494453
Japan,126919659,377915,0.0029775923050659944
Vietnam,94348835,331210,0.003510483197805251
United Kingdom,64088222,243610,0.0038011664608202113
Germany,80854408,357022,0.0044156157818878595
Italy,61855120,301340,0.004871706659044554
Uganda,37101745,241038,0.006496675560677807
Thailand,67976405,513120,0.007548501571979277


Above you can see the countries with the lowest area per person in ascending order.